<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/SBP-Brims/Notebooks/BERT_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs

In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.5 MB/s eta 0:00:00


In [2]:
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.7 MB/s eta 0:00:00


In [3]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install git+https://github.com/huggingface/accelerate
!pip uninstall -y transformers
!pip install transformers==4.28.0

Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: accelerate 0.20.3
Uninstalling accelerate-0.20.3:
  Successfully uninstalled accelerate-0.20.3
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-rwcyi3nk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-rwcyi3nk
  Resolved https://github.com/huggingface/accelerate to commit 95bffdec4326acf6a5d1c3dbaa857a26502aa265
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.21.0.dev0-py3-none-any.whl size=241741 sha256=2c9611728507941b6069adf3e93e3573a28843ab58d219c354db1d2fdc8130e0
  Stored in di

## Token hugging face

In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Mounting

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Dataset

In [7]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/MFC_prepared.csv', usecols=["code_frames", "annotations"], header=0)
df.code_frames = df.code_frames.astype(int)

label_dict = {}
for x in range(15):
    label_dict[x+1] = x

label_dict
df['label'] = df.code_frames.replace(label_dict)
df

df = pd.DataFrame(df)
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2)

# dataset = dataset.rename_column("code_frames", "label")
dataset = dataset.rename_column("annotations", "text")

dataset

DatasetDict({
    train: Dataset({
        features: ['code_frames', 'text', 'label'],
        num_rows: 37439
    })
    test: Dataset({
        features: ['code_frames', 'text', 'label'],
        num_rows: 9360
    })
})

## Prep

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased",
                                          do_lower_case=False)


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,
    max_length=128)

In [ ]:
mfc_tokenized = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/37439 [00:00<?, ? examples/s]

Map:   0%|          | 0/9360 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label, label2id = {},{}
for x in range(15):
    id2label[x] = x
        # {0: "NEGATIVE", 1: "POSITIVE"}
    label2id[x] = x
        # {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=15, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

## Training

In [ ]:
training_args = TrainingArguments(
    output_dir="Bert-trial",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mfc_tokenized["train"],
    eval_dataset=mfc_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/content/Bert-trial is already a clone of https://huggingface.co/jmLuis/Bert-trial. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.056600,0.934590,0.716453
2,0.774700,0.878050,0.732372
3,0.539900,0.896228,0.740705
4,0.383200,0.974471,0.744444
5,0.276500,1.087449,0.739637
6,0.200400,1.166801,0.742521
7,0.158200,1.229310,0.746261
8,0.130000,1.311914,0.744338
9,0.107100,1.349917,0.748504
10,0.090800,1.373488,0.747329


TrainOutput(global_step=11700, training_loss=0.38943232919415854, metrics={'train_runtime': 8952.2159, 'train_samples_per_second': 41.821, 'train_steps_per_second': 1.307, 'total_flos': 1.7475327949322484e+16, 'train_loss': 0.38943232919415854, 'epoch': 10.0})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/May23_06-27-47_b0d428ad89d4/events.out.tfevents.1684823269.b0d428ad89d4.195.0:   0%|         …

To https://huggingface.co/jmLuis/Bert-trial
   e6654cd..3bf1cfe  main -> main

   e6654cd..3bf1cfe  main -> main

To https://huggingface.co/jmLuis/Bert-trial
   3bf1cfe..57ac3af  main -> main

   3bf1cfe..57ac3af  main -> main



'https://huggingface.co/jmLuis/Bert-trial/commit/3bf1cfedf8342de533dac196663939f14f2e4824'

# Prediction

In [8]:
from transformers import AutoModelForSequenceClassification
import torch
from transformers import AutoTokenizer


In [9]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
text = 'Immigrants without HOPE need help entering college'
text = 'Nan'

In [10]:
tokenizer = AutoTokenizer.from_pretrained("jmLuis/Bert-trial")
inputs = tokenizer(text, return_tensors="pt")

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("jmLuis/Bert-trial")
with torch.no_grad():
    logits = model(**inputs).logits

In [12]:
predicted_class_id = logits.argmax().item()
out = model.config.id2label[predicted_class_id]
out+1

15

In [13]:
tokenizer = AutoTokenizer.from_pretrained("jmLuis/Bert-trial")
model = AutoModelForSequenceClassification.from_pretrained("jmLuis/Bert-trial")


def infer_bert(row):
    inputs = tokenizer(str(row['Text']), return_tensors="pt",  truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    out = model.config.id2label[predicted_class_id]
    return int(out)+1  #adding onw because original label is not zero index, however training is.

## applying in curated dataset

In [ ]:
import pandas as pd
# curated_df = pd.read_csv(f'{dir}Official/curated_Dataset.csv', usecols=["Annotate", "Text"], header=0)
curated_df = pd.read_csv(f'{dir}BERT-curated/Bert_curated_dataset-55.csv', usecols=["Annotate", "Text"], header=0)

curated_df

,Annotate,Text
0,2.0,"DOH logs 1,920 new COVID-19 cases, highest dai..."
1,14.0,"The Philippines on Sunday registered 1,920 ne..."
2,11.0,"According to GMA Integrated News Research, th..."
3,9.0,"It is the fourth straight day of more than 1,..."
4,9.0,Data from the Department of Health's COVID-19...
...,...,...
132092,NaN,"""The Philippines and China also discussed new ..."
132093,NaN,"On Thursday, Lazaro started chairing with Chin..."
132094,NaN,"The conduct of the BCM, a confidence-building ..."
132095,NaN,"In an earlier pronouncement, Lazaro asserted t..."


In [ ]:
curated_df.shape

(132097, 2)

In [ ]:
# Original

# save=0
# for i, row in curated_df.iterrows():
#     curated_df.at[i,'Annotate'] = infer_bert(row)
#     if(i%100==0):
#         print(i)
#     if (i%2000==0):
#         print('saved', save)
#         curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-{save}.csv', index=False)
#         save+=1


In [ ]:
# Token indices sequence length is longer than the specified maximum sequence length for this model (642 > 512). Running this sequence through the model will result in indexing errors
# RuntimeError: The size of tensor a (642) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
# curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-last.csv', index=False)

In [ ]:
# curated_df.iloc[:29953].to_csv(f'{dir}BERT-curated/Bert_curated_dataset-cut.csv', index=False)

In [ ]:
save=60
for i, row in curated_df.iloc[120000:].iterrows():
    curated_df.at[i,'Annotate'] = infer_bert(row)
    if(i%100==0):
        print(i)
    if (i%2000==0):
        print('saved', save)
        curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-{save}.csv', index=False)
        save+=1


120000
saved 60
120100
120200
120300
120400
120500
120600
120700
120800
120900
121000
121100
121200
121300
121400
121500
121600
121700
121800
121900
122000
saved 61
122100
122200
122300
122400
122500
122600
122700
122800
122900
123000
123100
123200
123300
123400
123500
123600
123700
123800
123900
124000
saved 62
124100
124200
124300
124400
124500
124600
124700
124800
124900
125000
125100
125200
125300
125400
125500
125600
125700
125800
125900
126000
saved 63
126100
126200
126300
126400
126500
126600
126700
126800
126900
127000
127100
127200
127300
127400
127500
127600
127700
127800
127900
128000
saved 64
128100
128200
128300
128400
128500
128600
128700
128800
128900
129000
129100
129200
129300
129400
129500
129600
129700
129800
129900
130000
saved 65
130100
130200
130300
130400
130500
130600
130700
130800
130900
131000
131100
131200
131300
131400
131500
131600
131700
131800
131900
132000
saved 66


In [ ]:
curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-last.csv', index=False)

In [ ]:
curated_df.tail()

,Annotate,Text
132092,14.0,"""The Philippines and China also discussed new ..."
132093,14.0,"On Thursday, Lazaro started chairing with Chin..."
132094,8.0,"The conduct of the BCM, a confidence-building ..."
132095,14.0,"In an earlier pronouncement, Lazaro asserted t..."
132096,8.0,This came as China is accused of conducting gr...


## Predict using hedlines and lead MFC

In [ ]:
import pandas as pd
curated_df = pd.read_csv(f'{dir}Official/curated_Dataset_headlines_lead.csv', header=0)

curated_df

,Text,Annotate
0,"DOH logs 1,920 new COVID-19 cases, highest dai...",NaN
1,"The Philippines on Sunday registered 1,920 new...",NaN
2,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",NaN
3,The Department of Health said Saturday that th...,NaN
4,Bettor wins P55.3 million jackpot in Grand Lot...,NaN
...,...,...
21599,The lower chamber recently gave its final appr...,NaN
21600,Immediate expansion of gov’t subsidies to kind...,NaN
21601,A network of advocates called for the immediat...,NaN
21602,"P​H, China vow UNCLOS​ compliance, seek Indo-P...",NaN


In [ ]:
save=0
for i, row in curated_df.iterrows():
    curated_df.at[i,'Annotate'] = infer_bert(row)
    if(i%100==0):
        print(i)
    if (i%2000==0):
        print('saved', save)
        curated_df.to_csv(f'{dir}predictions-HL/BERT/Bert_curated_dataset-{save}.csv', index=False)
        save+=1


0
saved 0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
saved 1
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
saved 2
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
saved 3
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
saved 4
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
saved 5
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
saved 6
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
saved 7
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
1750

In [ ]:
curated_df.to_csv(f'{dir}predictions-HL/BERT/Bert_curated_dataset-last.csv', index=False)

# Others

In [ ]:
#REference fine tuning
# https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb